<a href="https://colab.research.google.com/github/Kh-Harakeh/OS-Banker-s-Algorithm/blob/main/Banker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Part I**

In [11]:
import threading
from threading import Lock
import time

class Process:
    def __init__(self, id, max_resources, allocated_resources):
        self.id = id
        self.max_resources = max_resources
        self.allocated_resources = allocated_resources
        self.need_resources = [max_resources[i] - allocated_resources[i] for i in range(len(max_resources))]
        self.lock = Lock()

class Resource:
    def __init__(self, total_resources):
        self.total_resources = total_resources
        self.available_resources = total_resources[:]  # Initialize available_resources with total_resources
        self.lock = Lock()

def is_safe_state(processes, available_resources):
    work = available_resources[:]
    finish = [False] * len(processes)
    safe_sequence = []

    while True:
        allocated = False
        for i, process in enumerate(processes):
            if not finish[i] and all(process.need_resources[j] <= work[j] for j in range(len(work))):
                for j in range(len(work)):
                    work[j] += process.allocated_resources[j]
                safe_sequence.append(process.id)
                finish[i] = True
                allocated = True
                break
        if not allocated:
            break

    return all(finish), safe_sequence

def request_resources(process, request, processes, resources):
    with process.lock, resources.lock:
        if all(request[i] <= process.need_resources[i] for i in range(len(request))) and all(request[i] <= resources.available_resources[i] for i in range(len(request))):
            for i in range(len(request)):
                resources.available_resources[i] -= request[i]
                process.allocated_resources[i] += request[i]
                process.need_resources[i] -= request[i]

            safe, safe_sequence = is_safe_state(processes, resources.available_resources)

            if safe:
                return True, safe_sequence
            else:
                for i in range(len(request)):
                    resources.available_resources[i] += request[i]
                    process.allocated_resources[i] -= request[i]
                    process.need_resources[i] += request[i]

    return False, []

def process_thread(process, request, processes, resources):
    granted, safe_sequence = request_resources(process, request, processes, resources)
    if granted:
        print(f"Process {process.id} request granted. Safe sequence: {safe_sequence}")
    else:
        print(f"Process {process.id} request denied. System would be in an unsafe state.")

def race_condition_monitor(processes, resources, check_interval=1):
    while True:
        with resources.lock:
            for process in processes:
                with process.lock:
                    if any(process.allocated_resources[i] > process.max_resources[i] for i in range(len(process.max_resources))):
                        print(f"Error: Process {process.id} is allocated more resources than it needs.")

                    safe, _ = is_safe_state(processes, resources.available_resources)
                    if not safe:
                        print("Alert: The system is in an unsafe state.")

        time.sleep(check_interval)

def display_state(processes, available_resources):
    print(f"Available Resources: {available_resources}")
    for process in processes:
        print(f"Process {process.id}: Max: {process.max_resources}, Allocated: {process.allocated_resources}, Need: {process.need_resources}")

def main():
    total_resources = [10, 5, 7]
    available_resources = [3, 3, 5]
    resources = Resource(available_resources)
    processes = [
        Process(0, [7, 5, 3], [0, 1, 0]),
        Process(1, [3, 2, 2], [2, 0, 0]),
        Process(2, [9, 0, 2], [3, 0, 2]),
        Process(3, [2, 2, 2], [2, 1, 1]),
        Process(4, [4, 3, 3], [0, 0, 2])
    ]

    display_state(processes, resources.available_resources)

    # Start the race condition monitor in a separate thread
    monitor_thread = threading.Thread(target=race_condition_monitor, args=(processes, resources))
    monitor_thread.daemon = True
    monitor_thread.start()

    # Handle a single input for resource request
    try:
        process_id = int(input("Enter process ID to request resources: "))
        request = list(map(int, input("Enter resource request (space-separated): ").split()))

        process = next((p for p in processes if p.id == process_id), None)
        if process is None:
            print(f"Error: Process {process_id} not found.")
        else:
            t = threading.Thread(target=process_thread, args=(process, request, processes, resources))
            t.start()
            t.join()

            display_state(processes, resources.available_resources)
    except ValueError:
        print("Invalid input. Please enter integers only.")

if __name__ == "__main__":
    main()


Available Resources: [3, 3, 5]
Process 0: Max: [7, 5, 3], Allocated: [0, 1, 0], Need: [7, 4, 3]
Process 1: Max: [3, 2, 2], Allocated: [2, 0, 0], Need: [1, 2, 2]
Process 2: Max: [9, 0, 2], Allocated: [3, 0, 2], Need: [6, 0, 0]
Process 3: Max: [2, 2, 2], Allocated: [2, 1, 1], Need: [0, 1, 1]
Process 4: Max: [4, 3, 3], Allocated: [0, 0, 2], Need: [4, 3, 1]
Enter process ID to request resources: 2
Enter resource request (space-separated): 6 0 0
Process 2 request denied. System would be in an unsafe state.
Available Resources: [3, 3, 5]
Process 0: Max: [7, 5, 3], Allocated: [0, 1, 0], Need: [7, 4, 3]
Process 1: Max: [3, 2, 2], Allocated: [2, 0, 0], Need: [1, 2, 2]
Process 2: Max: [9, 0, 2], Allocated: [3, 0, 2], Need: [6, 0, 0]
Process 3: Max: [2, 2, 2], Allocated: [2, 1, 1], Need: [0, 1, 1]
Process 4: Max: [4, 3, 3], Allocated: [0, 0, 2], Need: [4, 3, 1]


## **Part II**

In [12]:
import threading
from threading import Lock
import time
import random

class Process:
    def __init__(self, id, max_resources, allocated_resources):
        self.id = id
        self.max_resources = max_resources
        self.allocated_resources = allocated_resources
        self.need_resources = [max_resources[i] - allocated_resources[i] for i in range(len(max_resources))]
        self.lock = Lock()

class Resource:
    def __init__(self, total_resources):
        self.total_resources = total_resources
        self.available_resources = total_resources[:]
        self.lock = Lock()

class DynamicResource(Resource):
    def __init__(self, total_resources):
        super().__init__(total_resources)

    def add_resources(self, additional_resources):
        with self.lock:
            for i in range(len(additional_resources)):
                self.total_resources[i] += additional_resources[i]
                self.available_resources[i] += additional_resources[i]

def is_safe_state(processes, available_resources):
    work = available_resources[:]
    finish = [False] * len(processes)
    safe_sequence = []

    while True:
        allocated = False
        for i, process in enumerate(processes):
            if not finish[i] and all(process.need_resources[j] <= work[j] for j in range(len(work))):
                for j in range(len(work)):
                    work[j] += process.allocated_resources[j]
                safe_sequence.append(process.id)
                finish[i] = True
                allocated = True
                break
        if not allocated:
            break

    return all(finish), safe_sequence

def request_resources(process, request, processes, resources):
    with process.lock, resources.lock:
        if all(request[i] <= process.need_resources[i] for i in range(len(request))) and all(request[i] <= resources.available_resources[i] for i in range(len(request))):
            for i in range(len(request)):
                resources.available_resources[i] -= request[i]
                process.allocated_resources[i] += request[i]
                process.need_resources[i] -= request[i]

            safe, safe_sequence = is_safe_state(processes, resources.available_resources)

            if safe:
                return True, safe_sequence
            else:
                for i in range(len(request)):
                    resources.available_resources[i] += request[i]
                    process.allocated_resources[i] -= request[i]
                    process.need_resources[i] += request[i]

    return False, []

def process_thread(process, request, processes, resources):
    granted, safe_sequence = request_resources(process, request, processes, resources)
    if granted:
        print(f"Process {process.id} request granted. Safe sequence: {safe_sequence}")
    else:
        print(f"Process {process.id} request denied. System would be in an unsafe state.")

def race_condition_monitor(processes, resources, check_interval=1):
    while True:
        with resources.lock:
            for process in processes:
                with process.lock:
                    if any(process.allocated_resources[i] > process.max_resources[i] for i in range(len(process.max_resources))):
                        print(f"Error: Process {process.id} is allocated more resources than it needs.")

                    safe, _ = is_safe_state(processes, resources.available_resources)
                    if not safe:
                        print("Alert: The system is in an unsafe state.")

        time.sleep(check_interval)

def display_state(processes, available_resources):
    print(f"Available Resources: {available_resources}")
    for process in processes:
        print(f"Process {process.id}: Max: {process.max_resources}, Allocated: {process.allocated_resources}, Need: {process.need_resources}")

def dynamic_resource_changer(dynamic_resource, interval):
    while True:
        new_resources = [random.randint(0, 5) for _ in range(len(dynamic_resource.total_resources))]
        print(f"Resources added: {new_resources}")
        dynamic_resource.add_resources(new_resources)
        print(f"New total resources: {dynamic_resource.total_resources}")
        print(f"New available resources: {dynamic_resource.available_resources}")
        time.sleep(interval)

def main():
    total_resources = [10, 5, 7]
    available_resources = [3, 3, 5]

    dynamic_resource = DynamicResource(total_resources)
    dynamic_resource.available_resources = available_resources

    processes = [
        Process(0, [7, 5, 3], [0, 1, 0]),
        Process(1, [3, 2, 2], [2, 0, 0]),
        Process(2, [9, 0, 2], [3, 0, 2]),
        Process(3, [2, 2, 2], [2, 1, 1]),
        Process(4, [4, 3, 3], [0, 0, 2])
    ]

    display_state(processes, dynamic_resource.available_resources)

    # Start the race condition monitor in a separate thread
    monitor_thread = threading.Thread(target=race_condition_monitor, args=(processes, dynamic_resource))
    monitor_thread.daemon = True
    monitor_thread.start()

    # Start the dynamic resource changer in a separate thread
    changer_thread = threading.Thread(target=dynamic_resource_changer, args=(dynamic_resource, 5))
    changer_thread.daemon = True
    changer_thread.start()

    # Handle a single input for resource request
    try:
        process_id = int(input("Enter process ID to request resources: "))
        request = list(map(int, input("Enter resource request (space-separated): ").split()))

        process = next((p for p in processes if p.id == process_id), None)
        if process is None:
            print(f"Error: Process {process_id} not found.")
        else:
            t = threading.Thread(target=process_thread, args=(process, request, processes, dynamic_resource))
            t.start()
            t.join()

            display_state(processes, dynamic_resource.available_resources)
    except ValueError:
        print("Invalid input. Please enter integers only.")

if __name__ == "__main__":
    main()

Available Resources: [3, 3, 5]
Process 0: Max: [7, 5, 3], Allocated: [0, 1, 0], Need: [7, 4, 3]
Process 1: Max: [3, 2, 2], Allocated: [2, 0, 0], Need: [1, 2, 2]
Process 2: Max: [9, 0, 2], Allocated: [3, 0, 2], Need: [6, 0, 0]
Process 3: Max: [2, 2, 2], Allocated: [2, 1, 1], Need: [0, 1, 1]
Process 4: Max: [4, 3, 3], Allocated: [0, 0, 2], Need: [4, 3, 1]
Resources added: [1, 2, 1]
New total resources: [11, 7, 8]
New available resources: [4, 5, 6]
Resources added: [5, 1, 0]
New total resources: [16, 8, 8]
New available resources: [9, 6, 6]
Resources added: [3, 1, 5]
New total resources: [19, 9, 13]
New available resources: [12, 7, 11]
Resources added: [4, 4, 0]
New total resources: [23, 13, 13]
New available resources: [16, 11, 11]
Resources added: [4, 4, 1]
New total resources: [27, 17, 14]
New available resources: [20, 15, 12]
Resources added: [1, 1, 3]
New total resources: [28, 18, 17]
New available resources: [21, 16, 15]


KeyboardInterrupt: Interrupted by user

## **Part III**

In [14]:
import threading
from threading import Lock
import time
import random

class Process:
    def __init__(self, id, max_resources, allocated_resources):
        self.id = id
        self.max_resources = max_resources
        self.allocated_resources = allocated_resources
        self.need_resources = [max_resources[i] - allocated_resources[i] for i in range(len(max_resources))]
        self.lock = Lock()

class Resource:
    def __init__(self, total_resources):
        self.total_resources = total_resources
        self.available_resources = total_resources[:]
        self.lock = Lock()

class DynamicResource(Resource):
    def __init__(self, total_resources):
        super().__init__(total_resources)

    def add_resources(self, additional_resources):
        with self.lock:
            for i in range(len(additional_resources)):
                self.total_resources[i] += additional_resources[i]
                self.available_resources[i] += additional_resources[i]

def is_safe_state(processes, available_resources):
    work = available_resources[:]
    finish = [False] * len(processes)
    safe_sequence = []

    while True:
        allocated = False
        for i, process in enumerate(processes):
            if not finish[i] and all(process.need_resources[j] <= work[j] for j in range(len(work))):
                for j in range(len(work)):
                    work[j] += process.allocated_resources[j]
                safe_sequence.append(process.id)
                finish[i] = True
                allocated = True
                break
        if not allocated:
            break

    return all(finish), safe_sequence

def request_resources(process, request, processes, resources):
    with process.lock, resources.lock:
        if all(request[i] <= process.need_resources[i] for i in range(len(request))) and all(request[i] <= resources.available_resources[i] for i in range(len(request))):
            for i in range(len(request)):
                resources.available_resources[i] -= request[i]
                process.allocated_resources[i] += request[i]
                process.need_resources[i] -= request[i]

            safe, safe_sequence = is_safe_state(processes, resources.available_resources)

            if safe:
                return True, safe_sequence
            else:
                for i in range(len(request)):
                    resources.available_resources[i] += request[i]
                    process.allocated_resources[i] -= request[i]
                    process.need_resources[i] += request[i]

    return False, []

def process_thread(process, request, processes, resources):
    granted, safe_sequence = request_resources(process, request, processes, resources)
    if granted:
        print(f"Process {process.id} request granted. Safe sequence: {safe_sequence}")
    else:
        print(f"Process {process.id} request denied. System would be in an unsafe state.")

def race_condition_monitor(processes, resources, check_interval=1):
    while True:
        with resources.lock:
            for process in processes:
                with process.lock:
                    if any(process.allocated_resources[i] > process.max_resources[i] for i in range(len(process.max_resources))):
                        print(f"Error: Process {process.id} is allocated more resources than it needs.")

                    safe, _ = is_safe_state(processes, resources.available_resources)
                    if not safe:
                        print("Alert: The system is in an unsafe state.")

        time.sleep(check_interval)

def display_state(processes, available_resources):
    print(f"Available Resources: {available_resources}")
    for process in processes:
        print(f"Process {process.id}: Max: {process.max_resources}, Allocated: {process.allocated_resources}, Need: {process.need_resources}")

def dynamic_resource_changer(dynamic_resource, interval):
    while True:
        new_resources = [random.randint(0, 5) for _ in range(len(dynamic_resource.total_resources))]
        print(f"Resources added: {new_resources}")
        dynamic_resource.add_resources(new_resources)
        print(f"New total resources: {dynamic_resource.total_resources}")
        print(f"New available resources: {dynamic_resource.available_resources}")
        time.sleep(interval)

def main():
    total_resources = [10, 5, 7]
    available_resources = [3, 3, 5]

    dynamic_resource = DynamicResource(total_resources)
    dynamic_resource.available_resources = available_resources

    processes = [
        Process(0, [7, 5, 3], [0, 1, 0]),
        Process(1, [3, 2, 2], [2, 0, 0]),
        Process(2, [9, 0, 2], [3, 0, 2]),
        Process(3, [2, 2, 2], [2, 1, 1]),
        Process(4, [4, 3, 3], [0, 0, 2])
    ]

    display_state(processes, dynamic_resource.available_resources)

    # Start the race condition monitor in a separate thread
    monitor_thread = threading.Thread(target=race_condition_monitor, args=(processes, dynamic_resource))
    monitor_thread.daemon = True
    monitor_thread.start()

    # Start the dynamic resource changer in a separate thread
    changer_thread = threading.Thread(target=dynamic_resource_changer, args=(dynamic_resource, 5))
    changer_thread.daemon = True
    changer_thread.start()

    # Parallel processing of resource requests
    requests = [
        (processes[0], [0, 2, 0]),
        (processes[4], [0, 3, 0]),
        (processes[1], [1, 0, 2]),
        (processes[3], [0, 1, 0])
    ]

    threads = []
    for process, request in requests:
        t = threading.Thread(target=process_thread, args=(process, request, processes, dynamic_resource))
        t.start()
        threads.append(t)

    for t in threads:
        t.join()

    display_state(processes, dynamic_resource.available_resources)

if __name__ == "__main__":
    main()

Available Resources: [3, 3, 5]
Process 0: Max: [7, 5, 3], Allocated: [0, 1, 0], Need: [7, 4, 3]
Process 1: Max: [3, 2, 2], Allocated: [2, 0, 0], Need: [1, 2, 2]
Process 2: Max: [9, 0, 2], Allocated: [3, 0, 2], Need: [6, 0, 0]
Process 3: Max: [2, 2, 2], Allocated: [2, 1, 1], Need: [0, 1, 1]
Process 4: Max: [4, 3, 3], Allocated: [0, 0, 2], Need: [4, 3, 1]
Resources added: [4, 5, 3]
New total resources: [14, 10, 10]
New available resources: [7, 8, 8]
Process 0 request granted. Safe sequence: [0, 1, 2, 3, 4]
Process 4 request granted. Safe sequence: [0, 1, 2, 3, 4]
Process 1 request granted. Safe sequence: [1, 0, 2, 3, 4]
Process 3 request granted. Safe sequence: [1, 0, 2, 3, 4]
Available Resources: [6, 2, 6]
Process 0: Max: [7, 5, 3], Allocated: [0, 3, 0], Need: [7, 2, 3]
Process 1: Max: [3, 2, 2], Allocated: [3, 0, 2], Need: [0, 2, 0]
Process 2: Max: [9, 0, 2], Allocated: [3, 0, 2], Need: [6, 0, 0]
Process 3: Max: [2, 2, 2], Allocated: [2, 2, 1], Need: [0, 0, 1]
Process 4: Max: [4, 3, 3]